<a href="https://colab.research.google.com/github/SAYANSAMANTA2005/2d-vector-using-vector-of-vector/blob/master/Text_Classification_Using_BERT_%26_Tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
df = pd.read_csv("/content/drive/MyDrive/Dataset/spam.csv")
df

,category,message,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN
...,...,...,...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...,NaN,NaN,NaN
5568,ham,Will Ì_ b going to esplanade fr home?,NaN,NaN,NaN
5569,ham,"Pity, * was in mood for that. So...any other s...",NaN,NaN,NaN
5570,ham,The guy did some bitching but I acted like i'd...,NaN,NaN,NaN


In [ ]:
df.drop (['Unnamed: 2','Unnamed: 3','Unnamed: 4'],axis =1,inplace = True)


df_ham =df[df["category"]=="ham"]
df_spam = df[df["category"]=="spam"]

df_ham_downsampled = df_ham.sample(df_spam.shape[0])
#df_ham_downsampled

df_balanced = pd.concat([df_spam,df_ham_downsampled])


df_balanced["spam"] = df_balanced["category"].apply(lambda x: 1 if x =="spam" else 0)
#df_balanced

from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test  = train_test_split(df_balanced["message"],df_balanced["spam"],test_size = 0.2,stratify = df_balanced["spam"])

In [ ]:
!pip install tensorflow_hub
!pip install tensorflow_text

In [ ]:
!pip install --upgrade tensorflow
!pip install --upgrade tensorflow_hub tensorflow_text

In [ ]:
import tensorflow_hub as hub
import tensorflow_text as text
import tensorflow as tf

DOWNLOADING PRE-TRAINED BERT MODEL

In [ ]:
# Load BERT preprocessing model
bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")

# Load BERT encoder model
bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")

In [ ]:
def get_sentence_embedding(sentences):
  preprocessed_text = bert_preprocess(sentences)
  return bert_encoder(preprocessed_text)["pooled_output"]

get_sentence_embedding(["500$ discount.hurry up",
                     "when will you eat food?"])

<tf.Tensor: shape=(2, 768), dtype=float32, numpy=
array([[-0.8435168 , -0.5132724 , -0.8884571 , ..., -0.7474884 ,
        -0.7531473 ,  0.91964483],
       [-0.89923227, -0.35631353, -0.54270977, ..., -0.3208228 ,
        -0.76459616,  0.93686306]], dtype=float32)>

In [ ]:
e = get_sentence_embedding([
    "banana",
    "grapes",
    "mango",
    "tiger",
    "cat",
    "lion",
    "elon musk",
    "bill gates",
    "jeff bezoes"

])

In [ ]:
e

<tf.Tensor: shape=(9, 768), dtype=float32, numpy=
array([[-0.7606919 , -0.14219387,  0.49604604, ...,  0.42165306,
        -0.5322141 ,  0.80312175],
       [-0.86023223, -0.2124296 ,  0.49156845, ...,  0.39797997,
        -0.60506296,  0.8447167 ],
       [-0.71288633, -0.15463911,  0.38401708, ...,  0.35278746,
        -0.50991315,  0.73474103],
       ...,
       [-0.7504134 , -0.2681262 , -0.2668972 , ...,  0.02839348,
        -0.5938098 ,  0.7974987 ],
       [-0.78544426, -0.29949754,  0.41027164, ...,  0.52225286,
        -0.49573594,  0.8150757 ],
       [-0.8913701 , -0.5605981 , -0.92077786, ..., -0.4725777 ,
        -0.670285  ,  0.8698612 ]], dtype=float32)>

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_similarity([e[3]],[e[5]])

array([[0.9934568]], dtype=float32)

In [ ]:
# bert layers
text_input = tf.keras.layers.Input(shape = (),dtype = tf.string,name = "text")

# Specify the output shape for the Lambda layer
preprocessed_text = tf.keras.layers.Lambda(bert_preprocess, output_shape=(128,))(text_input)
outputs = bert_encoder(preprocessed_text)
# neural network layers

l = tf.keras.layers.Dropout(0.1,name = "droupout")(outputs["pooled_output"])
l = tf.keras.layers.Dense(1,activation = "sigmoid",name ="output")(l)

# construct a final model

model = tf.keras.Model(inputs = [text_input],outputs =[l])

ValueError: Exception encountered when calling layer 'keras_layer_1' (type KerasLayer).

A KerasTensor is symbolic: it's a placeholder for a shape an a dtype. It doesn't have any actual numerical value. You cannot convert it to a NumPy array.

Call arguments received by layer 'keras_layer_1' (type KerasLayer):
  • inputs=<KerasTensor shape=(None, 128), dtype=float32, sparse=False, name=keras_tensor_2>
  • training=None

In [ ]:
# bert layers
text_input = tf.keras.layers.Input(shape = (),dtype = tf.string,name = "text")

# Specify the output shape for the Lambda layer
preprocessed_text = tf.keras.layers.Lambda(bert_preprocess, output_shape=(128,))(text_input)
outputs = bert_encoder(preprocessed_text)

# neural network layers

l = tf.keras.layers.Dropout(0.1,name = "droupout")(outputs["pooled_output"])
l = tf.keras.layers.Dense(1,activation = "sigmoid",name ="output")(l)

# construct a final model

model = tf.keras.Model(inputs = [text_input],outputs =[l])

ValueError: Exception encountered when calling layer 'keras_layer_1' (type KerasLayer).

A KerasTensor is symbolic: it's a placeholder for a shape an a dtype. It doesn't have any actual numerical value. You cannot convert it to a NumPy array.

Call arguments received by layer 'keras_layer_1' (type KerasLayer):
  • inputs=<KerasTensor shape=(None, 128), dtype=float32, sparse=False, name=keras_tensor_1>
  • training=None

In [ ]:
def build_model():
    text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name="text")
    preprocessed_text = bert_preprocess(text_input)
    outputs = bert_encoder(preprocessed_text)

    x = tf.keras.layers.Dropout(0.1, name='dropout')(outputs['pooled_output'])
    x = tf.keras.layers.Dense(1, activation='sigmoid', name='output')(x)

    model = tf.keras.Model(inputs=[text_input], outputs=[x])
    return model

In [ ]:
build_model()

ValueError: Exception encountered when calling layer 'keras_layer' (type KerasLayer).

A KerasTensor is symbolic: it's a placeholder for a shape an a dtype. It doesn't have any actual numerical value. You cannot convert it to a NumPy array.

Call arguments received by layer 'keras_layer' (type KerasLayer):
  • inputs=<KerasTensor shape=(None,), dtype=string, sparse=None, name=text>
  • training=None

In [ ]:
def build_model():
    text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name="text")
    preprocessed_text = bert_preprocess(text_input)
    outputs = bert_encoder(preprocessed_text)

    x = tf.keras.layers.Dropout(0.1, name='dropout')(outputs['pooled_output'])
    x = tf.keras.layers.Dense(1, activation='sigmoid', name='output')(x)

    model = tf.keras.Model(inputs=[text_input], outputs=[x])
    return model
    model.compile(optimizer = "adam",loss = "binary_crossentropy",metrics = ["accuracy"])

model =build_model()

ValueError: Exception encountered when calling layer 'keras_layer' (type KerasLayer).

A KerasTensor is symbolic: it's a placeholder for a shape an a dtype. It doesn't have any actual numerical value. You cannot convert it to a NumPy array.

Call arguments received by layer 'keras_layer' (type KerasLayer):
  • inputs=<KerasTensor shape=(None,), dtype=string, sparse=None, name=text>
  • training=None

In [ ]:
# Input layer
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name="text")

# Preprocessing (replace with actual preprocessing logic)
preprocessed_text = bert_preprocess(text_input)


# BERT encoding (replace with actual BERT encoding logic)
outputs = bert_encoder(preprocessed_text)

# Dense layer
l = tf.keras.layers.Droupout(0.1,name = "dropout")(outputs["pooled_output"])
l = tf.keras.layers.Dense(1, activation='sigmoid',name = "output")(l) # Example dense layer

# Model creation
model = tf.keras.Model(inputs=text_input, outputs= [l])


ValueError: Exception encountered when calling layer 'keras_layer' (type KerasLayer).

A KerasTensor is symbolic: it's a placeholder for a shape an a dtype. It doesn't have any actual numerical value. You cannot convert it to a NumPy array.

Call arguments received by layer 'keras_layer' (type KerasLayer):
  • inputs=<KerasTensor shape=(None,), dtype=string, sparse=None, name=text>
  • training=None

In [ ]:
# Input layer
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name="text")

# BERT encoding (replace with actual BERT encoding logic)
outputs = bert_encoder(text_input)  # Pass the KerasTensor directly to bert_encoder

# Dense layer
l = tf.keras.layers.Dropout(0.1, name="dropout")(outputs["pooled_output"])
l = tf.keras.layers.Dense(1, activation='sigmoid', name="output")(l)  # Example dense layer

# Model creation
model = tf.keras.Model(inputs=text_input, outputs=[l])

ValueError: Exception encountered when calling layer 'keras_layer_1' (type KerasLayer).

A KerasTensor is symbolic: it's a placeholder for a shape an a dtype. It doesn't have any actual numerical value. You cannot convert it to a NumPy array.

Call arguments received by layer 'keras_layer_1' (type KerasLayer):
  • inputs=<KerasTensor shape=(None,), dtype=string, sparse=None, name=text>
  • training=None

In [ ]:
# Input layer
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name="text")

# Preprocessing (replace with actual preprocessing logic)
preprocessed_text = bert_preprocess(text_input) # Preprocess the input text here

# BERT encoding
outputs = bert_encoder(preprocessed_text)  # Pass preprocessed text to bert_encoder

# Dense layer
l = tf.keras.layers.Dropout(0.1, name="dropout")(outputs["pooled_output"])
l = tf.keras.layers.Dense(1, activation='sigmoid', name="output")(l)

# Model creation
model = tf.keras.Model(inputs=text_input, outputs=[l])

ValueError: Exception encountered when calling layer 'keras_layer' (type KerasLayer).

A KerasTensor is symbolic: it's a placeholder for a shape an a dtype. It doesn't have any actual numerical value. You cannot convert it to a NumPy array.

Call arguments received by layer 'keras_layer' (type KerasLayer):
  • inputs=<KerasTensor shape=(None,), dtype=string, sparse=None, name=text>
  • training=None